# EDA
En este notebook se hará un análisis exploratorio para poder tener una información mínima de los datos y se planteará como objetivo la reducción de la cantidad de datos debido a que se hara un deploy en la capa gratuita de Render y esta no cuenta con una gran catidad de recursos a nivel computacional.

## Análisis del archivo steam_games

In [134]:
import pandas as pd
import numpy as np
import gzip
import re
import ast

In [135]:
with gzip.open(r'E:\Data Science\PI1\Datasets\steam_games.json.gz','rt',encoding='utf-8') as f:
    df_games= pd.read_json(f, lines=True)

In [136]:
df_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


Se ve primero un vistazo general del dataframe y se evaluará que columnas son utiles para el MVP.

In [137]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


Como primer paso se eliminan los datos que no aportan nada de información.

In [138]:
df_games.drop(['publisher','url','specs','early_access','reviews_url'],axis=1,inplace=True)

Se ve como alrededor del 25% del total de entradas tienen información, por lo que se eliminan las filas vacias a continuación.

In [139]:
df_games=df_games.dropna(how='all')

Veremos ahora la cantidad de nulos

In [140]:
df_games.isnull().sum()

genres          3283
app_name           2
title           2050
release_date    2067
tags             163
price           1377
id                 2
developer       3299
dtype: int64

Primero se va a eliminar las filas donde los campos tengan pocos datos faltantes como *app_name*,*tags* e *id*.

In [141]:
df_games.dropna(subset=['app_name','tags','id'],inplace=True)

Por simplicidad y debido a que es un MVP se borrarán los nulos de las columnas *genres*, *release_date*, *price* y *developer*.  

In [142]:
df_games.dropna(subset=['genres','release_date','price','developer'],inplace=True)

Se verifica a continuación el estado del daframe para verificar si sigue habiendo datos nulos.

In [143]:
df_games.isnull().sum()

genres          0
app_name        0
title           0
release_date    0
tags            0
price           0
id              0
developer       0
dtype: int64

Ahora se comparan las columnas de *app_name* y *title* para ver su similitud y se tiene datos redundantes.

In [144]:
df_games[df_games['app_name']!=df_games['title']]

,genres,app_name,title,release_date,tags,price,id,developer
88390,"[Action, Adventure]",Sam & Max 101: Culture Shock,Sam &amp; Max 101: Culture Shock,2006-10-17,"[Point & Click, Comedy, Adventure, Detective, ...",19.99,8200.0,Telltale Games
88393,"[Action, Adventure]",Sam & Max 102: Situation: Comedy,Sam &amp; Max 102: Situation: Comedy,2006-12-20,"[Adventure, Action]",19.99,8210.0,Telltale Games
88419,[Strategy],Command & Conquer: Red Alert 3,Command &amp; Conquer: Red Alert 3,2008-10-28,"[Strategy, RTS, Base Building, Multiplayer, Co...",19.99,17480.0,EA Los Angeles
88492,[Strategy],Heroes of Might & Magic V: Hammers of Fate,Heroes of Might &amp; Magic V: Hammers of Fate,2006-11-14,"[Strategy, Turn-Based Strategy, Turn-Based, Fa...",9.99,15380.0,Nival
88494,[Strategy],Heroes of Might & Magic V: Tribes of the East,Heroes of Might &amp; Magic V: Tribes of the East,2007-10-16,"[Strategy, Turn-Based Strategy, Turn-Based, Fa...",9.99,15370.0,Nival
...,...,...,...,...,...,...,...,...
120181,"[Action, Adventure]",Sam & Max 105: Reality 2.0,Sam &amp; Max 105: Reality 2.0,2007-03-29,"[Adventure, Action]",19.99,8240.0,Telltale Games
120182,"[Action, Adventure]",Sam & Max 104: Abe Lincoln Must Die!,Sam &amp; Max 104: Abe Lincoln Must Die!,2007-02-22,"[Adventure, Action, Point & Click, Free to Pla...",19.99,8230.0,Telltale Games
120183,"[Action, Adventure]",Sam & Max 106: Bright Side of the Moon,Sam &amp; Max 106: Bright Side of the Moon,2007-04-26,"[Adventure, Action]",19.99,8250.0,Telltale Games
120208,"[Action, Strategy]",Making History: The Calm & the Storm,Making History: The Calm &amp; the Storm,2007-03-13,"[Strategy, Action, Turn-Based Strategy, Grand ...",4.99,6250.0,Muzzy Lane


Se va a eliminar la columna *title* debido a la cantidad de erroes basado solo con una simple inspección visual

In [145]:
df_games.drop('title',axis=1,inplace=True)

In [146]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27449 entries, 88310 to 120443
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        27449 non-null  object 
 1   app_name      27449 non-null  object 
 2   release_date  27449 non-null  object 
 3   tags          27449 non-null  object 
 4   price         27449 non-null  object 
 5   id            27449 non-null  float64
 6   developer     27449 non-null  object 
dtypes: float64(1), object(6)
memory usage: 1.7+ MB


Se verificara que no haya duplicados en las columnas

In [147]:
df_games['id'].duplicated().sum()

1

In [148]:
df_games.drop_duplicates(subset='id',inplace=True)

In [149]:
df_games['app_name'].duplicated().sum()

18

In [150]:
df_games.drop_duplicates(subset='app_name',inplace=True)

In [151]:
df_games['app_name'].duplicated().sum()

0

Se modificará el campo *release_date* ya que solo se necesita el año por lo que se procede a crear una función para modificar esta columna

In [152]:
#ESe eliminan los valores SOON, SOON™ y coming soon de la columna 'release_date' que son juegos que están por salir
# Se crea una lista con los valores a eliminar
valores = ['SOON', 'SOON™','coming soon']

# Eliminar las filas donde 'release_date' es 'SOON' o 'SOON™'
df_games = df_games[~df_games['release_date'].isin(valores)]

In [153]:
#Se crea un función para extraer los 4 primeros o ultimos caracteres de la columna 'release_date'
def extract_four_numbers(s):
    if re.match('^\\d{4}', s):
        return s[:4]  
    else:
        return s[-4:]  

# Aplica la función a la columna 'release_date' y crea una nueva columna 'release_year'
df_games['release_year'] = df_games['release_date'].apply(extract_four_numbers)

Se elimina la columna *release_date* ya que es innecesaria.

In [154]:
df_games.drop('release_date',axis=1,inplace=True)

Ahora se va a modificar la columna *price* para cambiar los strings de *free to play* a 0.

In [155]:
df_games['price'] = df_games['price'].apply(lambda x: 0 if isinstance(x, str) else x)

Se verifica que no haya mas strings.

In [156]:
df_games['price'].apply(type).value_counts()

price
<class 'float'>    27427
Name: count, dtype: int64

Se normanliza *app_name* y *developer* pcon la funcion title()

In [157]:
df_games['app_name']=df_games['app_name'].apply(lambda x: x.title())

In [158]:
df_games['developer']=df_games['developer'].apply(lambda x: x.title())

Se proceden a cambiar los tipos de datos para que esten acordes a los datos de las columnas.


In [159]:
#Primero se verifica el valor maximo de la columna id
df_games['id'].max()

2028850.0

In [166]:
df_games[['id', 'price']] = df_games[['id', 'price']].astype({'id': 'int32', 'price': 'float16'})

In [168]:
df_games['price']=df_games['price'].apply(lambda x: round(x,2))

Se guardan los datos limpios en formato parquet.

In [170]:
df_games.to_parquet(r'Datasets\steam_games_clean.parquet',index=False)